In [1]:
from data_preparation import *

In [53]:
import torch
import numpy as np
import torch.optim as optim
from sklearn.model_selection import KFold
from GSR_Net.preprocessing import *
from GSR_Net.model import *
from GSR_Net.train import *
import argparse
import torch
import random

seed_value = 42
random.seed(seed_value)  # Python's built-in random
np.random.seed(seed_value)  # NumPy
torch.manual_seed(seed_value)  # PyTorch
torch.cuda.manual_seed(seed_value)  # For CUDA
torch.cuda.manual_seed_all(seed_value)  # For multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
path_to_data = '/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/data'

x_train, x_test, y_train = load_data_tensor(path_to_data)

In [5]:
X, Y = x_train, y_train

In [34]:
class ModelArgs:  
    def __init__(self, epochs, lr, splits, lmbda, lr_dim, hr_dim, hidden_dim,
                 padding):
        self.epochs = epochs
        self.lr = lr
        self.splits = splits
        self.lmbda = lmbda
        self.lr_dim = lr_dim
        self.hr_dim = hr_dim
        self.hidden_dim = hidden_dim
        self.padding = padding

args = ModelArgs(
    epochs=200,
    lr=0.0001,
    splits=3,
    lmbda=16,
    lr_dim=160,
    #hr_dim=320,
    #hidden_dim=320,
    hr_dim=320,
    hidden_dim=320,
    padding=26,
)

In [11]:
cv = KFold(n_splits=args.splits, random_state=42, shuffle=True)
ks = [0.9, 0.7, 0.6, 0.5]


In [71]:
seed_value = 42
random.seed(seed_value)  # Python's built-in random
np.random.seed(seed_value)  # NumPy
torch.manual_seed(seed_value)  # PyTorch
torch.cuda.manual_seed(seed_value)  # For CUDA
torch.cuda.manual_seed_all(seed_value)  # For multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)
print(model)

GSRNet(
  (layer): GSRLayer()
  (net): GraphUnet(
    (start_gcn): GCN(
      (proj): Linear(in_features=160, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (bottom_gcn): GCN(
      (proj): Linear(in_features=320, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (end_gcn): GCN(
      (proj): Linear(in_features=640, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
  )
  (gc1): GraphConvolution()
  (gc2): GraphConvolution()
)


In [55]:
Y.size()

torch.Size([167, 268, 268])

In [14]:
def train_model():
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    for train_index, test_index in cv.split(X):
        subjects_adj, test_adj, subjects_ground_truth, test_ground_truth = X[
            train_index], X[test_index], Y[train_index], Y[test_index]
        train(model, optimizer, subjects_adj, subjects_ground_truth, args)
        test(model, test_adj, test_ground_truth, args)
    

In [8]:
from evaluations import *

In [72]:
saved_state_dict = torch.load('/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_models/vanilla_model.pth')
model.load_state_dict(saved_state_dict, strict=True)

<All keys matched successfully>

In [73]:
preds_list = []
model.eval()
with torch.no_grad():
    for test_data in x_test:
        pred, _, _, _ = model(test_data)
        pred = unpad(pred, args.padding)
        preds_list.append(pred)
        
pred_tensor = torch.stack(preds_list)

In [74]:
pred_tensor[0].max()

tensor([[[1.0000, 0.7764, 0.4353,  ..., 0.0998, 0.2571, 0.1263],
         [0.7764, 1.0000, 0.6378,  ..., 0.0980, 0.2364, 0.1093],
         [0.4353, 0.6378, 1.0000,  ..., 0.0469, 0.2029, 0.0302],
         ...,
         [0.0998, 0.0980, 0.0469,  ..., 1.0000, 0.4802, 0.2537],
         [0.2571, 0.2364, 0.2029,  ..., 0.4802, 1.0000, 0.4296],
         [0.1263, 0.1093, 0.0302,  ..., 0.2537, 0.4296, 1.0000]],

        [[1.0000, 0.8181, 0.4180,  ..., 0.0938, 0.1668, 0.1188],
         [0.8181, 1.0000, 0.3852,  ..., 0.1116, 0.2194, 0.1468],
         [0.4180, 0.3852, 1.0000,  ..., 0.0644, 0.1898, 0.0683],
         ...,
         [0.0938, 0.1116, 0.0644,  ..., 1.0000, 0.3785, 0.2183],
         [0.1668, 0.2194, 0.1898,  ..., 0.3785, 1.0000, 0.4294],
         [0.1188, 0.1468, 0.0683,  ..., 0.2183, 0.4294, 1.0000]],

        [[1.0000, 0.4691, 0.5693,  ..., 0.2313, 0.1520, 0.1883],
         [0.4691, 1.0000, 0.5372,  ..., 0.1575, 0.1392, 0.1954],
         [0.5693, 0.5372, 1.0000,  ..., 0.0484, 0.0264, 0.

In [22]:
pred_tensor.size()

torch.Size([112, 268, 268])

In [23]:
evaluate(pred_tensor, Y)

0

1

2

3

4

5

6

7

8

9

10

11



KeyboardInterrupt: 

In [15]:
pred = model(x_test[0])[0]

In [16]:
pred.size()

torch.Size([320, 320])

In [43]:
from data_preparation import *

In [60]:
def find_preds_and_convert_to_submission(file_name, model=model, test_set=x_test):
    preds_list = []
    with torch.no_grad():
        for test_data in test_set:
            pred, _, _, _ = model(test_data)
            pred = unpad(pred, args.padding)
            preds_list.append(pred)
    pred_tensor = torch.stack(preds_list).cpu().numpy()
    generate_submission_file(pred_tensor, f'/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/submission_files/{file_name}')        